In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.numpy_ops import np_config
housing = fetch_california_housing()

c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class MLPLinearRegressor(object):
    def __init__(self, lr=0.001, n_layers=3, n_neurons=3):
        self.graph = tf.Graph()
        self.lr = 0.001
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.initialized = False

    def loss(self):
        return 1/2 * tf.reduce_sum( tf.square((tf.reshape(self.predicted, [-1, 1]) - self.y)))

    def initialize(self, X, y):
        n_samples, n_features = X.shape
        n_output = y.shape[0]
        normal_initiliazer = tf.random_normal_initializer(seed=0, mean=0.0, stddev=1.0)
        XN = tf.keras.utils.normalize(X)
        self.X = tf.constant(XN, name="X", dtype=tf.double)
        self.y = tf.constant(np.array(y).reshape(-1, 1), name="y", dtype=tf.double)

        self.f_weights = tf.Variable(normal_initiliazer([self.n_neurons, n_samples, n_features], dtype=tf.double), name="f_weights", dtype=tf.double, trainable=True )
        self.h_weights = tf.Variable(normal_initiliazer([self.n_layers, self.n_neurons, n_samples, n_output], dtype=tf.double), name="h_weights", dtype=tf.double, trainable=True )
        self.bias = tf.Variable([0.01 for l in range(self.n_layers)], name="bias", dtype=tf.double, trainable=True )
        
        self.activation = tf.TensorArray(tf.double, size=0, dynamic_size=True, clear_after_read=False, tensor_array_name="Activation_Layers")
        self.activation.write(0, [tf.math.sigmoid(tf.matmul(X, tf.transpose(self.f_weights[n])) + self.bias[0]) for n in range(self.n_neurons)]).mark_used()
        for l in range(1,self.n_layers-1):
            self.activation.write(l, [tf.math.sigmoid(tf.matmul(self.activation.read(l-1)[n], self.h_weights[l][n]) + self.bias[l]) for n in range(self.n_neurons)]).mark_used()
        self.activation.write(self.n_layers-1, [tf.math.sigmoid(tf.matmul(self.activation.read(self.n_layers-2)[n], self.h_weights[self.n_layers-1][n]) + self.bias[self.n_layers-1]) for n in range(self.n_neurons)]).mark_used()

        #self.loss = lambda : tf.losses.mean_squared_error(self.predicted, y)
        self.learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(self.lr, 5, 0.85, name="Learning_Rate")
        self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)
        self.predicted = tf.Variable(tf.reduce_mean(tf.math.sigmoid(tf.tensordot(tf.reduce_mean(tf.reduce_mean(self.h_weights[self.n_layers-1], axis=0), axis=0), tf.reduce_mean(tf.reduce_mean(self.activation.read(self.n_layers-1), axis=0), axis=1), axes=0) + tf.reshape(self.bias[self.n_layers-1], [-1, 1])), axis=0), trainable=True,dtype=tf.double, name="Predicted")

        self.vars = [self.f_weights, self.h_weights, self.predicted]
        self.initialized = True

    def train(self, y, epochs=75):
        if self.initialized:
          for e in range(epochs):
            with tf.GradientTape(persistent=True) as tape:
                tape.watch(self.vars)
                loss = self.loss()
            
            grads = tape.gradient(loss, self.vars)
            print(grads)
            self.optimizer.apply_gradients(zip(grads, self.vars))
        #self.accuracy = tf.keras.metrics.Accuracy().update_state(self.y, self.predicted)
        self.error = tf.keras.metrics.MeanSquaredError().update_state(tf.reshape(tf.sigmoid(self.y), [1, -1]), self.predicted)
        print(self.error.numpy())
          
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.95)

model = MLPLinearRegressor()
model.initialize(X_train, y_train)
model.train(y_train, epochs=1)

[None, None, <tf.Tensor: shape=(1032,), dtype=float64, numpy=
array([-0.57045794, -2.77345856, -0.09445794, ..., -0.70745783,
       -1.5974579 , -3.05045767])>]
1.0
